## Tools for building Henderson's Mixed Model Equations for multiple traits without missing data


In [1]:
using PedModule 
using DataFrames

In [2]:
include("../../Module/solver.jl")

Gibbs (generic function with 3 methods)

### <font color="red"> types and functions below are same (or very similar) as before.</font>

In [3]:
function mkDict(a)
    aUnique = unique(a)
    d = Dict()
    names = Array(Any,size(aUnique,1))
    for (i,s) in enumerate(aUnique)
        names[i] = s
        d[s] = i
    end
    return d,names
end

function getData(trm,df::DataFrame,mme::MME)
    nObs = size(df,1)
    trm.str = Array(AbstractString,nObs)
    trm.val = Array(Float64,nObs)
    
    if trm.factors[1] == :intercept ##modified from Melanie's HW ##new
        str = fill(string(trm.factors[1]),nObs) 
        val = fill(1.0,nObs)
        trm.str = str
        trm.val = val
        return
    end
    
    myDf = df[trm.factors]   
        
    if trm.factors[1] in mme.covVec
        str = fill(string(trm.factors[1]),nObs)
        val = df[trm.factors[1]]
    else
        str = [string(i) for i in df[trm.factors[1]]]
        val = fill(1.0,nObs)
    end
    for i=2:trm.nFactors
        if trm.factors[i] in mme.covVec
            str = str .* fill(" x "*string(trm.factors[i]),nObs)
            val = val .* df[trm.factors[i]]
        else
            str = str .* fill(" x ",nObs) .* [string(j) for j in df[trm.factors[i]]]
            val = val .* fill(1.0,nObs)
        end
    end
    trm.str = str
    trm.val = val
end


function getNames(mme)
    names = Array(AbstractString,0)
    for trm in mme.modelTerms
        for name in trm.names
            push!(names,trm.trmStr*": "*name)
        end
    end
    return names
end  

function covList(mme::MME, covStr::AbstractString)
    covVec = split(covStr," ",keep=false) 
    mme.covVec = [symbol(i) for i in covVec]
    nothing
end

function getSolJ(mme::MME, df::DataFrame)
    if size(mme.mmeRhs)==() 
        getMME(mme,df)
    end
    p = size(mme.mmeRhs,1)
    return [getNames(mme) Jacobi(mme.mmeLhs,fill(0.0,p),mme.mmeRhs,0.3,tol=0.000001)]
end

function getSolG(mme::MME, df::DataFrame)
    if size(mme.mmeRhs)==() 
        getMME(mme,df)
    end
    p = size(mme.mmeRhs,1)
    return [getNames(mme) GaussSeidel(mme.mmeLhs,fill(0.0,p),mme.mmeRhs,tol=0.000001)]
end


function addA(mme::MME)
    pedTrmVec = mme.pedTrmVec
    for (i,trmi) = enumerate(pedTrmVec)
        pedTrmi  = mme.modelTermDict[trmi]
        startPosi  = pedTrmi.startPos
        endPosi    = startPosi + pedTrmi.nLevels - 1
        for (j,trmj) = enumerate(pedTrmVec)
            pedTrmj  = mme.modelTermDict[trmj]
            startPosj  = pedTrmj.startPos
            endPosj    = startPosj + pedTrmj.nLevels - 1  
            mme.mmeLhs[startPosi:endPosi,startPosj:endPosj] = 
            mme.mmeLhs[startPosi:endPosi,startPosj:endPosj] + mme.Ai*mme.Gi[i,j] 
        end
    end
end

LoadError: LoadError: UndefVarError: MME not defined
while loading In[3], in expression starting on line 12

### Types and Functions

In [4]:
type ModelTerm 
    iModel::Int64
    trmStr::AbstractString
    nFactors::Int64
    factors::Array{Symbol,1}
    str::Array{AbstractString,1}                    # used to store the data for this term as strings
    val::Array{Float64,1}
    startPos::Int64                         # start pos in HMME
    nLevels::Int64                           
    X::SparseMatrixCSC{Float64,Int64}
    names::Array{Any,1}
end

type MME
    modelVec::Array{AbstractString,1} #new
    modelTerms::Array{ModelTerm,1}    #new
    modelTermDict::Dict{AbstractString,ModelTerm} #new
    lhsVec::Array{Symbol,1}
    covVec::Array{Symbol,1}
    pedTrmVec::Array{AbstractString,1}
    mmeLhs
    mmeRhs
    ped
    Gi::Array{Float64,2}
    Ri::Array{Float64,2}
    Ai
    mmePos::Int64
end

function getTerm(trmStr,m)
    trm = ModelTerm(m,string(m)*":"*trmStr,0,[],[],[],0,0,spzeros(0,0),[])
    factorVec = split(trmStr,"*")
    trm.nFactors = length(factorVec)
    trm.factors = [symbol(strip(f)) for f in factorVec]
    return trm
end

function initMME(models::AbstractString,R::Array{Float64,2})
    # returns an MME object for muilding the mme corresponding 
    # to the input string
    if models==""
        error("modelEquation is empty\n")
    end
    modelVec = split(models,[';','\n']; keep=false)
    nModels  = size(modelVec,1)
    lhsVec   = Symbol[]
    modelTerms = ModelTerm[]
    dict = Dict{AbstractString,ModelTerm}()
    for (m,model) = enumerate(modelVec)
        lhsRhs = split(model,"=")
        lhsVec = [lhsVec,symbol(strip(lhsRhs[1]))]
        rhs = strip(lhsRhs[2])
        rhsVec = split(rhs,"+")    
        mTrms = [getTerm(strip(trmStr),m) for trmStr in rhsVec]
        modelTerms = [modelTerms; mTrms] #1:Animal 2:Animal
        for (i,trm) = enumerate(modelTerms) # this may need to change
            dict[trm.trmStr] = modelTerms[i]
        end 
    end
    Ri = inv(R)
    return MME(modelVec,modelTerms,dict,lhsVec,[],[],0,0,0,Array(Float64,1,1),Ri,0,1)
end 

function setAsRandom(mme::MME,randomStr::AbstractString,ped::PedModule.Pedigree, G::Array{Float64,2})
    pedTrmVec = split(randomStr," ";keep=false)
    res = []
    for trm in pedTrmVec
        for (m,model) = enumerate(mme.modelVec)
            strVec  = split(model,['=','+'])
            strpVec = [strip(i) for i in strVec]
            if trm in strpVec
                res = [res,string(m)*":"*trm]
            end
        end
    end
    mme.pedTrmVec = res
    mme.ped = ped
    mme.Gi = inv(G)
    nothing
end

function getMME(mme::MME, df::DataFrame)
    mme.mmePos = 1
    for trm in mme.modelTerms
        getData(trm,df,mme)
        getX(trm,mme)
    end
    n   = size(mme.modelTerms,1)
    trm = mme.modelTerms[1]
    X   = trm.X
    for i=2:n
        trm = mme.modelTerms[i]
        X = [X trm.X]
    end
    y = df[mme.lhsVec[1]]
    for i=2:size(mme.lhsVec,1)
        y    = [y, df[mme.lhsVec[i]] ] 
    end
    N  = size(y,1)
    ii = 1:N
    jj = fill(1,N)
    vv = y
    ySparse = sparse(ii,jj,vv)
    nObs = size(df,1)
    Ri = kron(mme.Ri,speye(nObs))
    mme.mmeLhs = X'Ri*X
    mme.mmeRhs = X'Ri*ySparse
    if mme.ped != 0
        ii,jj,vv = PedModule.HAi(mme.ped)
        HAi = sparse(ii,jj,vv)
        mme.Ai = HAi'HAi
        addA(mme::MME)
    end   
end

getFactor1(str) = [strip(i) for i in split(str,"x")][1]

function getX(trm,mme::MME)
    pedSize = 0
    nObs  = size(trm.str,1)
    if trm.trmStr in mme.pedTrmVec
        trm.names   = PedModule.getIDs(mme.ped)
        trm.nLevels = length(mme.ped.idMap)
        xj = round(Int64,[mme.ped.idMap[getFactor1(i)].seqID for i in trm.str])
    else
        dict,trm.names  = mkDict(trm.str)
        trm.nLevels     = length(dict)
        xj    = round(Int64,[dict[i] for i in trm.str])
    end
    xi    = (trm.iModel-1)*nObs + collect(1:nObs)
    xv    = trm.val
    if mme.ped!=0
        pedSize = length(mme.ped.idMap)
        if trm.trmStr in mme.pedTrmVec
            # This is to ensure the X matrix for 
            # additive effect has the correct number of columns
            ii = 1         # adding a zero to
            jj = pedSize   # the last column in row 1
            vv = [0.0]
            xi = [xi;ii]
            xj = [xj;jj]
            xv = [xv;vv]
        end
    end 
    #make sure X has nObs*nModels rows
    nModels = size(mme.lhsVec,1)
    xi = [xi;1;nObs*nModels]
    xj = [xj;1;1]
    xv = [xv;0;0]
    trm.X = sparse(xi,xj,xv)
    trm.startPos = mme.mmePos
    mme.mmePos  += trm.nLevels
end

getX (generic function with 1 method)

In [5]:
ped = PedModule.mkPed("../../data/small.ped")
ped.idMap

Dict{Any,Any} with 5 entries:
  "O1" => PedModule.PedNode(3,"S1","D1",0.0)
  "S1" => PedModule.PedNode(1,"0","0",0.0)
  "O3" => PedModule.PedNode(4,"S1","D1",0.0)
  "D1" => PedModule.PedNode(2,"0","0",0.0)
  "O2" => PedModule.PedNode(5,"S1","D1",0.0)

In [6]:
ID = ["S1","D1","O1","O3"]
y1 = [100.0, 50.0, 150.0, 40.0]
y2 = [10.0,  12.9, 13.0,  5.0]
dfMT = DataFrame(Animal=ID,y1=y1,y2=y2)

,Animal,y1,y2
1,S1,100.0,10.0
2,D1,50.0,12.9
3,O1,150.0,13.0
4,O3,40.0,5.0


### Multiple Trait Animal Model 

In [7]:
models = "y1 = intercept + Animal;
          y2 = intercept + Animal"
R = [10 2; 2 1.0]
mme = initMME(models,R)
G0 = [5 1;1 1.0]
setAsRandom(mme,"Animal", ped,G0)

In [8]:
mme.pedTrmVec

 in depwarn at deprecated.jl:73
 in oldstyle_vcat_warning at /Applications/Julia-0.4.0.app/Contents/Resources/julia/lib/julia/sys.dylib
 in vect at abstractarray.jl:38
 in initMME at In[4]:51
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /Users/rohan/.julia/v0.4/IJulia/src/execute_request.jl:177
 in eventloop at /Users/rohan/.julia/v0.4/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:447
while loading In[7], in expression starting on line 4
 in depwarn at deprecated.jl:73
 in oldstyle_vcat_warning at /Applications/Julia-0.4.0.app/Contents/Resources/julia/lib/julia/sys.dylib
 in vect at abstractarray.jl:38
 in initMME at In[4]:51
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /Users/rohan/.julia/v0.4/IJulia/src/execute_request.jl:177
 in eventloop at /Users/rohan/.julia/v0.4/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:447
while loading In[7], in expression starting on line 4


2-element Array{AbstractString,1}:
 "1:Animal"
 "2:Animal"

In [9]:
getMME(mme,dfMT);

LoadError: LoadError: UndefVarError: getData not defined
while loading In[9], in expression starting on line 1

In [10]:
[trm.startPos for trm in mme.modelTerms]

4-element Array{Any,1}:
 0
 0
 0
 0

In [11]:
round(full(mme.mmeLhs),2)

LoadError: LoadError: MethodError: `full` has no method matching full(::Int64)
while loading In[11], in expression starting on line 1

In [12]:
getSolG(mme, dfMT)

LoadError: LoadError: UndefVarError: getSolG not defined
while loading In[12], in expression starting on line 1

###  Multiple Trait Model with Direct and Maternal Effects

In [13]:
dfMTMat = [dfMT[3:4,:]  DataFrame(mat = ["D1","D1"])]

,Animal,y1,y2,mat
1,O1,150.0,13.0,D1
2,O3,40.0,5.0,D1


In [14]:
models = "y1 = intercept + Animal + mat;
          y2 = intercept + Animal"
R = [10 2; 2 1.0]
mme = initMME(models,R)
G0 = [5   1    0.1
      1   1    0.01
      0.1 0.01 0.5]
setAsRandom(mme,"Animal mat", ped,G0)
mme.pedTrmVec

3-element Array{AbstractString,1}:
 "1:Animal"
 "2:Animal"
 "1:mat"   

In [15]:
getSolG(mme, dfMTMat)

LoadError: LoadError: UndefVarError: getSolG not defined
while loading In[15], in expression starting on line 1